# Building good training sets - Data preprocessing
数据的质量以及包含有用信息的数量是决定学习算法性能的关键因素之一，因此在给学习算法输入数据之前做一些数据检查和预处理是非常必要的。这部分包含的topics如下：
* 1.为数据集删除或增补缺失值
* 2.将类别数据形式化后用于机器学习算法
* 3.在构建模型时选取丰富的特征

## 1. Dealing with missing data
在实际应用中我们的样本会因为各种原因缺失一个或者更多的值，可能是数据收集过程中的一个错误，不合理的固定度量，调查中特定区域的简写留白，例如在一些表格中我们在缺失值的地方留一个空白，或者放一个填充字符串(NaN).大多数计算工具没有能力对缺失值进行处理，如果我们忽略缺失值，往往会得到不尽人意的结果。
## Eliminating samples anf features with missing values
处理缺失值最简单一种方法就是，删除整个数据集中缺失值对应的特征（columns）和样本（rows）。利用pandas中的dropna()方法很容易的实现。删除缺失值的方法虽然简单但是存在很大的缺点，当我们删除了大量的样本会使得结果可信度变差，删除特征有可能损失对分类非常有效的信息，使得分类的准确度降低。

In [46]:
# example for missing data of CSV data format
import pandas as pd
from io import StringIO

csv_data = """A,B,C,D
1.0, 2.0, 3.0, 4.0
5.0, 6.0, , 8.0
0.0, 11.0, 12.0, """
# csv_data = unicode(csv_data)
df = pd.read_csv(StringIO(csv_data),keep_default_na=False,na_values=[' '])
print(df,'\n',df.values)
print(df.isnull().sum())

     A     B     C    D
0  1.0   2.0   3.0  4.0
1  5.0   6.0   NaN  8.0
2  0.0  11.0  12.0  NaN 
 [[  1.   2.   3.   4.]
 [  5.   6.  nan   8.]
 [  0.  11.  12.  nan]]
A    0
B    0
C    1
D    1
dtype: int64


In [47]:
# Eliminate samples and features with missing values
print(df.dropna())
# Eliminate the columns with at least one NaN 
print(df.dropna(axis= 1))

#only drop rows where all columns are NaN
print(df.dropna(how= 'all'))

# drop rows that have not at least 4 non-NaN values
print(df.dropna(thresh= 4))

# only drop rows where NaN appear in specific column (here: 'C')
print(df.columns)
print(df.dropna(subset=['C']))

     A    B    C    D
0  1.0  2.0  3.0  4.0
     A     B
0  1.0   2.0
1  5.0   6.0
2  0.0  11.0
     A     B     C    D
0  1.0   2.0   3.0  4.0
1  5.0   6.0   NaN  8.0
2  0.0  11.0  12.0  NaN
     A    B    C    D
0  1.0  2.0  3.0  4.0
Index(['A', 'B', 'C', 'D'], dtype='object')
     A     B     C    D
0  1.0   2.0   3.0  4.0
2  0.0  11.0  12.0  NaN


## imputing missing values
缺失值常用的插补技巧：mean imputation，median，most-frequent

In [48]:
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis= 0)
imputed_data = imr.fit_transform(df)
print(imputed_data)

[[  1.    2.    3.    4. ]
 [  5.    6.    7.5   8. ]
 [  0.   11.   12.    6. ]]


## 2. Handing categorical data
类别数据特征分为：标称型（nominal）和顺序型（ordinal）

In [62]:
# create hybridly categorical dataset
df = pd.DataFrame([
    ['gree','M',12,'class1'],
    ['red','XL',8,'class2'],
    ['blue', 'L',7,'class3']
])
df.columns = ['color','size','price','classlabel']
print(df)

# Mapping original features
# 为了让学习算法可以正确解释原始特征，需要将categorical string 转换为 integer
# 定义 XL= L + 1 = M + 2
size_mapping = {'XL':3, 'L':2, 'M':1}
df['size'] = df['size'].map(size_mapping)
print(df)

# Encoding class label
import numpy as np

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
print(class_mapping)
df['classlabel'] = df['classlabel'].map(class_mapping)
print(df)
# converted class labels back to the original string representation
invert_class_mapping = {value:label for label, value in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(invert_class_mapping)
print('\n',df)

# using the scikit-learn function to encoding the label
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
print('\n',y)
print('\n',class_le.inverse_transform(y))

# Encoding nominal features
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
X = df[['color','size','price']].values
X[:, 0] = class_le.fit_transform(X[:,0])
numerical_feature = ohe.fit_transform(X).toarray()
print('\n',numerical_feature)

# method of get_dummies implemented in pandas
print(pd.get_dummies(df[['color','size','price']]))

  color size  price classlabel
0  gree    M     12     class1
1   red   XL      8     class2
2  blue    L      7     class3
  color  size  price classlabel
0  gree     1     12     class1
1   red     3      8     class2
2  blue     2      7     class3
{'class2': 1, 'class3': 2, 'class1': 0}
  color  size  price  classlabel
0  gree     1     12           0
1   red     3      8           1
2  blue     2      7           2

   color  size  price classlabel
0  gree     1     12     class1
1   red     3      8     class2
2  blue     2      7     class3

 [0 1 2]

 ['class1' 'class2' 'class3']

 [[  0.   1.   0.   1.  12.]
 [  0.   0.   1.   3.   8.]
 [  1.   0.   0.   2.   7.]]
   size  price  color_blue  color_gree  color_red
0     1     12           0           1          0
1     3      8           0           0          1
2     2      7           1           0          0


## 3. Bringing features onto the same scale
feature scaling是数据预处理过程中很关键的一步，很多机器学习都需要对特征进行归一化处理。而Decision tree 和Random forest这类算法，对特征多样性具有很好的鲁棒性不需要归一化处理。为什么要进行特征归一化？
* 1. 特征尺度相差很大时，容易受大尺度特征主导，影响分类的准确性（normalization feature scaling to range 0 to 1）,存在较大异常值时不适用。
$$ x_{sc}^{(i)} = \frac{x^{(i)} - x_{min}}{x_{max}- x_{min}}$$ 
* 2. 特征分布比较杂乱，容易受噪声，和异常点影响，通常要进行标准分布，如此可以减少噪声和异常值影响，加速学习（weights）。如标准正态分布，均匀分布等(standardization feature distribution to zero mean and standard deviation 1)
$$ x_{std}^{(i)} = \frac{x^{(i)} - \mu_{x}}{\sigma_{x}}$$

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
 

## 4. Selecting meaningful features
机器学习模型经常会遇到过拟合的问题，在训练集上性能表现良好，但不能很好的泛化到测试集上。防止过拟合常用的方法：
* 1.收集更多的训练数据
* 2.引入正则化项
* 3.选择简单一点的模型减少参数量
* 4.对数据进行降维处理

比较常用技巧的是2和4。
## Regularization
L1正则化：计算权重的绝对值求和，可以对特征进行选择，获得稀疏模型，对于large-scale问题可以极大的减少存储空间，加速训练过程。

L2正则化：计算权重平方和再开根号，可以防止过拟合，因为权重衰减的作用，使得训练过程稳定而快速。
## Sequential feature selection algorithm
减少模型计算复杂度和过拟合的一种可选择的方法是通过特征选择来降低维度，这种方法对没有正则化的模型非常有用。降低维度主要有两种技巧：特征选择（feature selection）和特征抽取（feature extraction），特征选择是选取原始特征的子集合，特征抽取是由原始特征集合构建新的特征空间。

连续特征选择算法属于贪婪搜索算法，用来将原始的$d$维特征减少到$k$维特征（$k<d$）.特征选择的目的是自动选择一个特征子集合，它与通过去除不相关特征和噪声来提高模型计算效率和减少泛化误差非常相关，这对不支持正则化的算法非常有用。一个典型的连续特征选择算法是**Sequential Backward Selection（SBS）**，主要目的是降低原始特征空间的维度使得模型性能最小下降的同时提高计算效率。特定情况下，如果模型过拟合的话，SBS可以提高模型的预测能力。